<a href="https://colab.research.google.com/github/malijarkas/ECSE551/blob/main/Personal_Mini_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Modules

In [1]:
# my imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score



# henry's import
!pip install numpy pandas matplotlib seaborn statsmodels scikit-learn autocorrect contractions tqdm spacy nltk colorama
!python -m spacy download en_core_web_sm


import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import os
from autocorrect import Speller
import contractions
import re
from tqdm import tqdm
from colorama import init, Fore, Style
from typing import Optional
import unicodedata
from difflib import SequenceMatcher

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger_eng')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_ch

True

### Mount drive and import data

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np

train_data_path = '/content/drive/My Drive/ECSE 551 Data Set/Train.csv'
test_data_path = '/content/drive/My Drive/ECSE 551 Data Set/Test.csv'

drive.mount('/content/drive')

input_df = pd.read_csv(train_data_path, header=None)
kaggle_df = pd.read_csv(test_data_path, header=None)


# Set the column names
input_df.columns = ['comments', 'classification']
kaggle_df.columns = ['id', 'comments']

# Get the unique classes
class_names = input_df['classification'].unique()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### alpha_Naive Bayes Classifier

TODO:
- fix the fixme
- another set of eyes to make sure its correct

In [11]:
class Naive_Bayes_Classifier:

  def __init__(self, num_features, num_classes):
    self.num_features = num_features
    self.num_classes = num_classes
    self.theta_k = np.zeros(num_classes)
    self.theta_j_k = np.zeros((num_classes, num_features))

  def train(self, X, Y, laplace_smoothing = True):
    num_features = self.num_features
    num_classes = self.num_classes

    num_samples = X.shape[0]

    class_frequency = np.zeros(num_classes)

    # k denotes the class number
    # j denotes the feature number
    theta_k = np.zeros(num_classes)
    theta_j_k = np.zeros((num_classes, num_features))

    # find frequency of output class
    for i in range (num_samples):
       for j in range (num_classes):
          class_index = int(Y[i])
          class_frequency[class_index] += 1
          break

    # find P(Y = k)
    for i in range (num_classes):
      theta_k[i] = class_frequency[i] / num_samples

    # find P(X=1|Y=k) = theta_j_k
    # P(X=0|Y=k) = 1 - P(X=1|Y=k) = 1 - theta_j_k
    for i in range (num_samples):
      # FIXME: used to be X[i][0, j] until it suddenly broke? change for kfold
      # FIX: assign to current_sample, not sure why X[i] = X[i].reshape doesnt work
      if X[i].ndim == 1:
        current_sample = X[i].reshape(1, -1)
      else:
        current_sample = X[i]
      for j in range (num_features):
        if (current_sample[0, j] == 1):
          class_index = int(Y[i])
          theta_j_k[class_index, j] += 1

    if laplace_smoothing:
      for i in range (num_classes):
        theta_j_k[i] = (theta_j_k[i] + 1) / (class_frequency[i] + 2)
    else:
      for i in range (num_classes):
        theta_j_k[i] = theta_j_k[i] / class_frequency[i]

    self.theta_k = theta_k
    self.theta_j_k = theta_j_k

  def predict(self, X):
    num_features = self.num_features
    num_classes = self.num_classes
    theta_k = self.theta_k
    theta_j_k = self.theta_j_k

    num_samples = X.shape[0]

    delta_k = np.zeros(num_classes)

    output_labels_indexed = np.zeros(num_samples, dtype=int)

    for i in range (num_samples):

      # calculate the delta_k for each class, iterating over the features
      for k in range (num_classes):
        delta_k[k] = np.log(theta_k[k])
        for j in range (num_features):
          if (X[i][0,j] == 1):
            delta_k[k] += np.log(theta_j_k[k][j])
          elif (X[i][0,j] == 0):
            delta_k[k] += np.log(1 - theta_j_k[k][j])

      #find the class with the highest delta_k
      max_delta_k_index = np.argmax(delta_k)
      output_labels_indexed[i] = int(max_delta_k_index)

    return output_labels_indexed

### Helper Functions

#### Misc functions

In [13]:
# writes string array to file with delimiter '\n'
def _write_to_csv(output, file_name):
  with open(file_name, 'w') as f:
    for i in range(len(output)):
      f.write(str(output[i]))
      f.write('\n')

def write_to_csv_kaggle(output, file_name):
  with open(file_name, 'w') as f:
    f.write("id,subreddit\n")
    for i in range(len(output)):
      f.write(str(i))
      f.write(',')
      f.write(str(output[i]))
      f.write('\n')

# Converts class names to indices, makes indexing in future operations easier
# Takes in the Y label numpy array and outputs it indexed
# The index values are dependant on the orders in class_names list
def class_name_to_index(class_names, Y_label):
    if not isinstance(Y_label, np.ndarray):
        raise TypeError("Y must be a numpy array")

    Y_string = np.copy(Y_label) #to avoid modifying Y_label
    Y_int = np.zeros(Y_string.size)

    class_dict = {name: idx for idx, name in enumerate(class_names)}
    for i in range(Y_string.size):
        if Y_string[i] in class_dict:
            Y_int[i] = class_dict[Y_string[i]]
        else:
            raise ValueError(f"Unexpected class name '{Y_string[i]}' in Y")
    return np.array(Y_int)

# Converts indices back to their class names
# Takes in the same list of class names and the numpy array Y_integer
def index_to_class_name(class_names, Y_index):
  if not isinstance(Y_index, np.ndarray):
    raise TypeError("Y_index must be a numpy array")

  Y_int = np.copy(Y_index)
  Y_string = np.empty(Y_int.size, dtype=object)

  if not (0 <= Y_int).all() and (Y_int < len(class_names)).all():
      raise ValueError("Y_index parameter contains invalid class index")

  for i in range(Y_int.size):
      Y_string[i] = class_names[int(Y_int[i])]
  return Y_string

# partitions, tokenizes, and train_test splits the data
# returns X_train_validate, X_test, Y_train_validate, Y_test to be input into kfold
# returns vectorizer for future tokenizing
def preprocess_data(input_df, num_features = 3000, ngram_range = (1,1)):
  from sklearn.naive_bayes import BernoulliNB
  from sklearn.model_selection import train_test_split
  from sklearn.feature_extraction.text import CountVectorizer

  # PLACEHOLDER begin until other preprocessing takes place
  input_df_processed = input_df.copy()
  # PLACEHOLDER end

  input_data_x_np = np.array(input_df_processed["comments"])
  input_data_y_np = np.array(input_df_processed["classification"])

  # initialize vectorizer
  vectorizer = CountVectorizer(max_features=num_features, binary=True, ngram_range = ngram_range)

  # convert input_data_x to binary
  input_data_x_bool_np = vectorizer.fit_transform(input_data_x_np).todense()

  # map input data y labels to indices
  input_data_y_indexed_np = class_name_to_index(class_names, input_data_y_np)

  X_train_validate, X_test, Y_train_validate, Y_test = train_test_split(input_data_x_bool_np, input_data_y_indexed_np, test_size=0.2, random_state=42)

  return X_train_validate, X_test, Y_train_validate, Y_test, vectorizer


#### K_fold related functions

In [5]:
# k fold splits the data
def _k_fold_split(input_data_x_b, input_data_y_indexed, k_folds = 10, random_state = 0):
  # k_fold
  kf = KFold(n_splits=k_folds, shuffle=True, random_state = random_state)

  # FIXME: is this the best way to intialize them?
  X_train = np.array([None] * k_folds)
  X_test = np.array([None] * k_folds)
  Y_train = np.array([None] * k_folds)
  Y_test = np.array([None] * k_folds)

  for i, (train_index, test_index) in enumerate(kf.split(input_data_x_b)):
    X_train[i], X_test[i] = np.array(input_data_x_b[train_index]), input_data_x_b[test_index]
    Y_train[i], Y_test[i] = input_data_y_indexed[train_index], input_data_y_indexed[test_index]

  print(X_train[0].shape)

  return X_train, X_test, Y_train, Y_test

def k_fold_loop(X_train_validate, Y_train_validate, k = 10, compare_w_scikit = True, verbose = True):

  scikit_classifiers = []
  NB_classifiers = []

  scikit_accuracies = np.zeros(k)
  NB_accuracies = np.zeros(k)

  # stores prediction labels for classifiers
  Y_pred = np.array([None] * k)
  if (compare_w_scikit):
    Y_pred_scikit = np.array([None] * k)

  X_train, X_validate, Y_train, Y_validate = _k_fold_split(X_train_validate, Y_train_validate, k_folds = k)

  for i in range (k):
  # initialize class, train, and predict
    NB = Naive_Bayes_Classifier(num_features=3000, num_classes=4)
    NB_classifiers.append(NB)
    NB_classifiers[i].train(X_train[i], Y_train[i])

    Y_pred[i] = (NB.predict(X_validate[i]))
    NB_accuracies[i] = accuracy_score(Y_validate[i], Y_pred[i])

    if (compare_w_scikit):
      CL = BernoulliNB()
      scikit_classifiers.append(CL)
      scikit_classifiers[i].fit(X_train[i], Y_train[i])

      Y_pred_scikit[i] = scikit_classifiers[i].predict(np.asarray(X_validate[i]))
      scikit_accuracies[i] = accuracy_score(Y_validate[i], Y_pred_scikit[i])
    if (verbose):
      print("Finished Test: " + str(i) + " with accuracy: " + str(NB_accuracies[i]))
  if (compare_w_scikit):
    return NB_classifiers, NB_accuracies, scikit_classifiers, scikit_accuracies
  return NB_classifiers, NB_accuracies



#### Henry's preprocess function

In [6]:
# Henry's function
class TextPreProcessor:
    def __init__(self, lowercase=True, remove_html=True, remove_urls=True, remove_emails=True,
                 normalize_unicode=True, remove_punctuation=True, remove_numbers=True,
                 remove_special_characters=True, remove_mentions_hashtags=True,
                 correct_spelling=False, expand_contractions=True, remove_duplicates=True,
                 remove_extra_whitespace=True, drop_missing=True, remove_short_comments=True,
                 min_length=5):
        self.lowercase = lowercase
        self.remove_html = remove_html
        self.remove_urls = remove_urls
        self.remove_emails = remove_emails
        self.normalize_unicode = normalize_unicode
        self.remove_punctuation = remove_punctuation
        self.remove_numbers = remove_numbers
        self.remove_special_characters = remove_special_characters
        self.remove_mentions_hashtags = remove_mentions_hashtags
        self.correct_spelling = correct_spelling
        self.expand_contractions = expand_contractions
        self.remove_duplicates = remove_duplicates
        self.remove_extra_whitespace = remove_extra_whitespace
        self.drop_missing = drop_missing
        self.remove_short_comments = remove_short_comments
        self.min_length = min_length
        self.spell = Speller()

    def preprocess(self, original_df, column='comments'):
        df = original_df.copy()

        # Wrapping each step in tqdm to show progress
        if self.lowercase:
            tqdm.pandas(desc="Lowercasing")
            df[column] = df[column].progress_apply(lambda x: x.lower() if isinstance(x, str) else x)

        if self.remove_html:
            tqdm.pandas(desc="Removing HTML tags")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'<.*?>', '', x) if isinstance(x, str) else x)

        if self.remove_urls:
            tqdm.pandas(desc="Removing URLs")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x) if isinstance(x, str) else x)

        if self.remove_emails:
            tqdm.pandas(desc="Removing Emails")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'\S+@\S+', '', x) if isinstance(x, str) else x)

        if self.normalize_unicode:
            tqdm.pandas(desc="Normalizing Unicode")
            df[column] = df[column].progress_apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8') if isinstance(x, str) else x)

        if self.remove_punctuation:
            tqdm.pandas(desc="Removing Punctuation")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)

        if self.remove_numbers:
            tqdm.pandas(desc="Removing Numbers")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)

        if self.remove_special_characters:
            tqdm.pandas(desc="Removing Special Characters")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x) if isinstance(x, str) else x)

        if self.remove_mentions_hashtags:
            tqdm.pandas(desc="Removing Mentions and Hashtags")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'@\w+|#\w+', '', x) if isinstance(x, str) else x)

        if self.correct_spelling:
            tqdm.pandas(desc="Correcting Spelling")
            df[column] = df[column].progress_apply(lambda x: self.spell(x) if isinstance(x, str) else x)

        if self.expand_contractions:
            tqdm.pandas(desc="Expanding Contractions")
            df[column] = df[column].progress_apply(lambda x: contractions.fix(x) if isinstance(x, str) else x)

        if self.remove_extra_whitespace:
            tqdm.pandas(desc="Removing Extra Whitespace")
            df[column] = df[column].progress_apply(lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x)

        if self.remove_duplicates:
            print("Removing Duplicates")
            df = df.drop_duplicates(subset=[column], keep='first')

        if self.drop_missing:
            print("Dropping Missing Values")
            df = df.dropna(subset=[column])

        if self.remove_short_comments:
            print("Removing Short Comments")
            df = df[df[column].str.len() > self.min_length]

        return df

#### Henry's post process function

In [7]:
class TextPostProcessor:
    def __init__(self, remove_stopwords=True, stem=True, lemmatize=True, pos_tag=True, ner=True, vectorize=True):
        """Initialize the text processor with desired processing steps."""
        self.remove_stopwords = remove_stopwords
        self.stem = stem
        self.lemmatize = lemmatize
        self.pos_tag = pos_tag
        self.ner = ner
        self.vectorize = vectorize

        # Initialize NLP tools
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
        self.nlp = spacy.load("en_core_web_sm")

    def _process_text(self, tokens):
        """Process a single text document through the pipeline."""
        if not tokens:
            return []

        # Convert tokens to text for spaCy processing
        text = ' '.join(tokens)
        doc = self.nlp(text)

        processed_tokens = tokens

        # Remove stopwords first to reduce processing load
        if self.remove_stopwords:
            processed_tokens = [token for token in processed_tokens
                              if token.lower() not in self.stop_words]

        # Lemmatize before stemming (if both are enabled)
        # This is more efficient as lemmatization can reduce the number of tokens
        if self.lemmatize:
            processed_tokens = [token.lemma_ for token in doc]

        # Apply stemming after lemmatization
        if self.stem:
            processed_tokens = [self.stemmer.stem(token) for token in processed_tokens]

        # POS tagging
        if self.pos_tag:
            processed_tokens = nltk.pos_tag(processed_tokens)
            # Extract just the tokens if we need to continue processing
            if self.ner:
                processed_tokens = [token for token, _ in processed_tokens]

        # Add NER results
        if self.ner:
            ner_entities = [ent.text for ent in doc.ents]
            processed_tokens.extend(ner_entities)

        return processed_tokens

    def postprocess(self, df, column='comments'):
        """Process all texts in the dataframe."""
        if not isinstance(df, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame")

        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in DataFrame")

        # Create a copy to avoid modifying the original
        processed_df = df.copy()

        # Process all comments
        tqdm.pandas(desc="Processing texts")
        processed_df[column] = processed_df[column].progress_apply(self._process_text)

        # Vectorize the processed comments
        if self.vectorize:
            tqdm.pandas(desc="Vectorizing")
            # Join tokens back into strings for vectorization
            text_for_vectorization = processed_df[column].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

            vectorizer = TfidfVectorizer()
            vectorized_comments = vectorizer.fit_transform(text_for_vectorization)

            # Store the vocabulary and feature names
            self.vectorizer_vocabulary_ = vectorizer.vocabulary_
            self.feature_names_ = vectorizer.get_feature_names_out()

            # Add vectorized results as a new column
            processed_df['vectorized_comments'] = list(vectorized_comments.toarray())

        return processed_df

### main

In [18]:
X_train_validate, X_test, Y_train_validate, Y_test, vectorizer = preprocess_data(input_df, 3000, ngram_range = (1,1))

NB_classifiers, NB_accuracies, scikit_classifiers, scikit_accuracies = k_fold_loop(X_train_validate, Y_train_validate)

(1008, 3000)
Finished Test: 0 with accuracy: 0.5625
Finished Test: 1 with accuracy: 0.4732142857142857
Finished Test: 2 with accuracy: 0.5089285714285714
Finished Test: 3 with accuracy: 0.48214285714285715
Finished Test: 4 with accuracy: 0.49107142857142855
Finished Test: 5 with accuracy: 0.49107142857142855
Finished Test: 6 with accuracy: 0.48214285714285715
Finished Test: 7 with accuracy: 0.42857142857142855
Finished Test: 8 with accuracy: 0.4375
Finished Test: 9 with accuracy: 0.5535714285714286


In [19]:
print("Accuracies ")

for i in range (len(NB_accuracies)):
  print("--------------------------------")
  print("NB accuracy for test " + str(i) + ": " + str(NB_accuracies[i]))
  print("Scikit accuracy for test " + str(i) + ": " + str(scikit_accuracies[i]))

print("**********************************")
print("Average NB accuracy", np.mean(NB_accuracies))
print("Average Scikit accuracy", np.mean(scikit_accuracies))

### ok, now retrain over test+validate data and predict over my test data and see if the accuracies match
Nb = Naive_Bayes_Classifier(num_features=3000, num_classes=4)

Nb.train(X_train_validate, Y_train_validate)
Y_pred_final = Nb.predict(X_test)
print("Final accuracy: " + str(accuracy_score(Y_test, Y_pred_final)))

Nb_sci = BernoulliNB()
Nb_sci.fit(np.asarray(X_train_validate), np.asarray(Y_train_validate))
Y_pred_final_sci = Nb_sci.predict(np.asarray(X_test))

print("Final scikit accuracy: " + str(accuracy_score(Y_test, Y_pred_final_sci)))


# kaggle_label_indexed = NB_classifiers[highest_accuracy_index].predict(test_data_kaggle_b)
# # kaggle_scikit_label_indexed = scikit_classifiers[highest_scikit_accuracy_index].predict(np.asarray(test_data_kaggle_b))

# kaggle_label = index_to_class_name(class_names, kaggle_label_indexed)
# # kaggle_scikit_label = index_to_class_name(class_names, kaggle_scikit_label_indexed)

# write_to_csv_kaggle(kaggle_label, "kaggle_predictions.csv")

#

# # ### these need to be reshuffled... thats why final predictions are low
# # ### ^^^ im not sure of the truth of this
# final_label = NB_classifiers[highest_accuracy_index].predict(input_data_x_b)
# final_scikit_label = scikit_classifiers[highest_scikit_accuracy_index].predict(np.asarray(input_data_x_b))
# print("final prediction using highest accuracy model")
# print("for my model:")
# print(calculate_accuracy(final_label, input_data_y_indexed))
# print("for scikit model:")
# print(calculate_accuracy(final_scikit_label, input_data_y_indexed))


Accuracies 
--------------------------------
NB accuracy for test 0: 0.5625
Scikit accuracy for test 0: 0.5625
--------------------------------
NB accuracy for test 1: 0.4732142857142857
Scikit accuracy for test 1: 0.4732142857142857
--------------------------------
NB accuracy for test 2: 0.5089285714285714
Scikit accuracy for test 2: 0.5089285714285714
--------------------------------
NB accuracy for test 3: 0.48214285714285715
Scikit accuracy for test 3: 0.48214285714285715
--------------------------------
NB accuracy for test 4: 0.49107142857142855
Scikit accuracy for test 4: 0.49107142857142855
--------------------------------
NB accuracy for test 5: 0.49107142857142855
Scikit accuracy for test 5: 0.49107142857142855
--------------------------------
NB accuracy for test 6: 0.48214285714285715
Scikit accuracy for test 6: 0.48214285714285715
--------------------------------
NB accuracy for test 7: 0.42857142857142855
Scikit accuracy for test 7: 0.42857142857142855
------------------

### old main

In [ ]:
from sklearn.naive_bayes import BernoulliNB

# preprocessor = TextPreProcessor(lowercase=True,
#                                 remove_html=1,
#                                 remove_urls=1,
#                                 remove_emails=1,
#                                 normalize_unicode=1,
#                                 remove_punctuation=1,
#                                 remove_numbers=1,
#                                 remove_special_characters=1,
#                                 remove_mentions_hashtags=1,
#                                 correct_spelling=0,
#                                 expand_contractions=1,
#                                 remove_duplicates=0,
#                                 remove_extra_whitespace=1,
#                                 drop_missing=0,
#                                 remove_short_comments=0, min_length=5,)

# postprocessor = TextPostProcessor(remove_stopwords=True, stem=True, lemmatize=False, pos_tag=False, ner=True, vectorize=True)


# processed_input_df = preprocessor.preprocess(train_df, column='comments')
# processed_test_df = preprocessor.preprocess(kaggle_test_df, column='comments')
# processed_input_df = postprocessor.postprocess(processed_input_df, column='comments')
# processed_input_df = train_df.copy()
# print(processed_input_df)


kaggle_df_processed = kaggle_df.copy()
X_kaggle_np= np.array(kaggle_df_processed["comments"][1:])


# convert input_data_x to binary

X_kaggle_bool_np = vectorizer.transform(X_kaggle_np).todense()

# run the main k fold train/test loop, compare with scikit naive bayes


# run the main k fold train/test loop, compare with scikit naive bayes






#### test_train_split main

In [ ]:
write_to_csv_kaggle(kaggle_label, "kaggle_predictions.csv")
print(kaggle_label)


In [ ]:
  NB = Naive_Bayes_Classifier(num_features=3000, num_classes=4)
  NB.train(X_train, Y_train)




In [ ]:
  NB.latest_output_indexed = NB.predict(X_test, Y_test)


In [ ]:
  NB.accuracy = calculate_accuracy(NB.latest_output_indexed, Y_test)
  print(NB.accuracy)

### Outdated

In [ ]:

# # converts class names to integers to be used as indices
# # takes in a list of class names and the numpy array Y
# def class_name_to_index(class_names, Y):
#     for i in range (Y.shape[0]):
#         for j in range (class_names.shape[0]):
#           if Y[i] == class_names[j]:
#             Y[i] = j
#             break
#           # elif j == class_names.shape[0] - 1:
#           #   print(Y[i])
#           #   print(j)
#           #   raise ValueError('unexpected class name in Y')
#     return Y

# # converts class index (i.e. integers) back to their class names
# # takes in the same list of class names and the numpy array Y_integer
# def index_to_class_name(class_names, Y_integer):
#     for i in range (Y_integer.shape[0]):
#         for j in range (class_names.shape[0]):
#           if Y_integer[i] == j:
#             Y_integer[i] = class_names[j]
#             break
#           elif j == class_names.shape[0] - 1:
#             raise ValueError('unexpected class index in Y_integer')
#     return Y_integer

# for i in range (test_data_y.shape[0]):
#   print(test_data_y[i])
#   if test_data_y[i] == 'Toronto':
#     Y_test[i] = 0
#   elif test_data_y[i] == 'Montreal':
#     Y_test[i] = 1
#   elif test_data_y[i] == 'London':
#     Y_test[i] = 2
#   elif test_data_y[i] == 'Brussels':
#     Y_test[i] = 3

